In [69]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, TreebankWordTokenizer, WhitespaceTokenizer
import numpy as np
import pandas as pd
import textwrap
from pprint import pprint
import os
import re
import data_processing
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
path_to_test_file = "acerpi_dataset/test/"
path_to_train_file = "acerpi_dataset/train/"

#### Open Annotated Sentences
We get the sentences from 'prep_data_labeling.ipynb' that we annotated fully manually with deccano.

In [5]:
train_documents = {}
for filename in os.listdir(path_to_train_file):
   if filename.endswith(".txt"):
      with open(os.path.join(path_to_train_file, filename), 'r', encoding="utf8") as f:
         key = int(filename.replace('.txt', ''))
         value = f.read()
         train_documents[key] = value

#### Open Unannotated Train Documents
We get the unnanotatted train texts and store each document as a key, value pair in a dictionary

In [224]:
annotated_sentences = pd.read_json(os.path.join(path_to_test_file, 'annotated_sentences.jsonl'), orient='record', lines=True)

In [225]:
annotated_sentences.head()

,id,text,document,sentence_id,duplicates,label
0,115,Documento gerado sob autenticação Nº LKB.506.4...,105798,0,[0],"[[233, 238, ORGANIZATION]]"
1,116,1/1 PORTARIA Nº 1955 de 05/03/2020 O PRÓ-REITO...,105798,1,[1],"[[72, 113, ORGANIZATION], [448, 475, MISCELLAN..."
2,117,MAURÍCIO VIÉGAS DA SILVA Pró-Reitor de Gestão ...,105798,2,"[2, 5, 8, 11, 17, 19, 21, 23, 25, 41, 56]","[[0, 24, PERSON], [25, 56, MISCELLANEOUS]]"
3,118,Documento gerado sob autenticação Nº BOA.507.6...,105799,3,[3],"[[233, 238, ORGANIZATION]]"
4,119,1/1 PORTARIA Nº 1956 de 05/03/2020 O PRÓ-REITO...,105799,4,[4],"[[72, 113, ORGANIZATION], [229, 235, MISCELLAN..."


Next we'll explode our dataframe so that duplicate sentences can become separate rows. We'll also drop a few columns that will have inconsistent information after the explode and will also not be useful for us.

In [226]:
pd.options.display.max_rows = 999
annotated_sentences = annotated_sentences.explode("duplicates")
annotated_sentences.drop(columns=['id', 'document', 'sentence_id'], inplace=True)
annotated_sentences.columns = ['text', 'sentence_id', 'label']
annotated_sentences.index = annotated_sentences['sentence_id']
annotated_sentences = annotated_sentences.sort_index()


In [227]:
tst_snt = annotated_sentences['text'].loc[1]
test_label = annotated_sentences['label'].loc[1]
print(annotated_sentences['text'].loc[1])
print(annotated_sentences['label'].loc[1])

1/1 PORTARIA Nº 1955 de 05/03/2020 O PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no uso de suas atribuições que lhe foram conferidas pela Portaria nº 7684, de 03 de outubro de 2016, do Magnífico Reitor, e conforme o Laudo Médico n°60131, RESOLVE Designar, temporariamente, nos termos da Lei nº 8.112, de 11 de dezembro de 1990, com redação dada pela Lei nº 9.527, de 10 de dezembro de 1997, a ocupante do cargo de ASSISTENTE EM ADMINISTRAÇÃO, do Quadro de Pessoal desta Universidade, FRANCIELE MARQUES ZIQUINATTI (Siape: 1091092 ), para substituir TURENE ANDRADE E SILVA NETO (Siape: 0356721 ), Diretor da Divisão de Preparo da Licitação do Departamento de Aquisição de Bens e Serviços, Código FG-4, em seu afastamento por motivo de Laudo Médico do titular da Função, no período de 30/01/2020 a 15/03/2020, com o decorrente pagamento das vantagens por 46 dias.
[[72, 113, 'ORGANIZATION'], [448, 475, 'MISCELLANEOUS'], [518, 546, 'PERSON'], [582, 609, 'PERSON'], [629

In [228]:
tokenized_sentences = []
token_positions = []
for idx, sentence in annotated_sentences['text'].items():
    tokenized_sentences.append(TreebankWordTokenizer().tokenize(sentence))
    token_positions.append(list(TreebankWordTokenizer().span_tokenize(sentence)))


#tst_words = TreebankWordTokenizer().tokenize(tst_snt)
#tst_pos = list(TreebankWordTokenizer().span_tokenize(tst_snt))
#print(tst_words)
#print(len(tst_words))
#print(tst_pos)

In [230]:
print(annotated_sentences['text'].loc[43])
print(tokenized_sentences[43])

1/2 PORTARIA Nº 3796 de 03/05/2017  Nomeia Coordenadoras do Programa Idiomas sem Fronteiras - IsF.
['1/2', 'PORTARIA', 'Nº', '3796', 'de', '03/05/2017', 'Nomeia', 'Coordenadoras', 'do', 'Programa', 'Idiomas', 'sem', 'Fronteiras', '-', 'IsF', '.']


In [231]:
#tst_tokens = list(zip(tst_words, tst_pos))
tokenized_sentence_position = []
for sentence, positions in zip(tokenized_sentences, token_positions):
    #print(list(zip(sentence, positions)))
    tokenized_sentence_position.append(list(zip(sentence, positions)))

In [233]:
print(annotated_sentences['label'].loc[43])
tokenized_sentence_position[43]

[[43, 91, 'MISCELLANEOUS']]


[('1/2', (0, 3)),
 ('PORTARIA', (4, 12)),
 ('Nº', (13, 15)),
 ('3796', (16, 20)),
 ('de', (21, 23)),
 ('03/05/2017', (24, 34)),
 ('Nomeia', (36, 42)),
 ('Coordenadoras', (43, 56)),
 ('do', (57, 59)),
 ('Programa', (60, 68)),
 ('Idiomas', (69, 76)),
 ('sem', (77, 80)),
 ('Fronteiras', (81, 91)),
 ('-', (92, 93)),
 ('IsF', (94, 97)),
 ('.', (97, 98))]

In [234]:
list_of_token_df = []
for list_of_words in tokenized_sentence_position:
    tokens_df = pd.DataFrame(list_of_words, columns = ['token', 'pos'])
    tokens_df[['start', 'end']] = tokens_df['pos'].to_list()
    tokens_df = tokens_df.drop(columns='pos')
    list_of_token_df.append(tokens_df)

In [235]:
list_of_token_df[43]

,token,start,end
0,1/2,0,3
1,PORTARIA,4,12
2,Nº,13,15
3,3796,16,20
4,de,21,23
5,03/05/2017,24,34
6,Nomeia,36,42
7,Coordenadoras,43,56
8,do,57,59
9,Programa,60,68


In [236]:
test_label

[[72, 113, 'ORGANIZATION'],
 [448, 475, 'MISCELLANEOUS'],
 [518, 546, 'PERSON'],
 [582, 609, 'PERSON'],
 [629, 636, 'MISCELLANEOUS'],
 [640, 671, 'ORGANIZATION'],
 [675, 719, 'ORGANIZATION']]

In [237]:
for index, token_df in enumerate(list_of_token_df):
    labels = []
    is_entity = False
    for idx, token in token_df.iterrows():
        for label in annotated_sentences['label'].loc[index]:
            if token['start'] == label[0]:
                labels.append('B-' + label[2])
                is_entity = True
                break
            if token['start'] >= label[0] and token['end'] <= label[1]:
                labels.append('I-' + label[2])
                is_entity = True
                break
            is_entity = False
        if is_entity == False:
            labels.append('O')
    token_df['label'] = labels

In [248]:
print(list_of_token_df[60])
annotated_sentences['text'][4]

                          token  start  end           label
0                     Documento      0    9               O
1                        gerado     10   16               O
2                           sob     17   20               O
3                  autenticação     21   33               O
4                            Nº     34   36               O
5               VOH.938.113.2VA     37   52               O
6                             ,     52   53               O
7                    disponível     54   64               O
8                            no     65   67               O
9                      endereço     68   76               O
10                         http     77   81               O
11                            :     81   82               O
12  //www.ufrgs.br/autenticacao     82  109               O
13                    Documento    110  119               O
14                  certificado    120  131               O
15              eletronicamente    132  

'1/1 PORTARIA Nº 1956 de 05/03/2020 O PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no uso de suas atribuições que lhe foram conferidas pela Portaria nº 7684, de 03 de outubro de 2016, do Magnífico Reitor, e conforme o Laudo Médico n°60170, RESOLVE Designar, temporariamente, nos termos da Lei nº 8.112, de 11 de dezembro de 1990, com redação dada pela Lei nº 9.527, de 10 de dezembro de 1997, o ocupante do cargo de TÉCNICO EM ASSUNTOS EDUCACIONAIS, do Quadro de Pessoal desta Universidade, FELIPE ANTÔNIO GUIDI SAUERESSIG (Siape: 1963089 ), para substituir JOSE EDUARDO HANDEL (Siape: 0357511 ), Secretário da COMGRAD de Nutrição, Código FG-7, em seu afastamento por motivo de Laudo Médico do titular da Função, no período de 05/02/2020 a 11/02/2020, com o decorrente pagamento das vantagens por 7 dias.'

In [191]:
list_of_token_df[0:3]

[                          token  start  end           label
 0                     Documento      0    9               O
 1                        gerado     10   16               O
 2                           sob     17   20               O
 3                  autenticação     21   33               O
 4                            Nº     34   36               O
 5               LKB.506.405.IRF     37   52               O
 6                             ,     52   53               O
 7                    disponível     54   64               O
 8                            no     65   67               O
 9                      endereço     68   76               O
 10                         http     77   81               O
 11                            :     81   82               O
 12  //www.ufrgs.br/autenticacao     82  109               O
 13                    Documento    110  119               O
 14                  certificado    120  131               O
 15              eletron

In [139]:
tokens_df

,token,start,end,label
0,1/1,0,3,O
1,PORTARIA,4,12,O
2,Nº,13,15,O
3,1955,16,20,O
4,de,21,23,O
5,05/03/2020,24,34,O
6,O,35,36,O
7,PRÓ-REITOR,37,47,O
8,DE,48,50,O
9,GESTÃO,51,57,O
